[Reference](https://medium.com/@Cartelis/launching-a-docker-based-modern-open-source-data-stack-a936b1bb0a43)

In [1]:
# Required python packages
from airflow.utils.dates import days_ago
from airflow.sensors.external_task_sensor import ExternalTaskSensor
from datetime import datetime
from datetime import timedelta
from airflow import DAG
from airflow.providers.airbyte.operators.airbyte import AirbyteTriggerSyncOperator
from airflow.providers.airbyte.sensors.airbyte import AirbyteJobSensor
from airflow_dbt.operators.dbt_operator import DbtRunOperator

# Arguments required by the DAG
default_args = {
    "owner": "airflow", # Owner of the dag
    "depends_on_past": False, # Dependence of previous attempts
    "retries": 1, # Limit of the number of retries
    "dir": "/opt/airflow/dbt/test_project", # Location of the dbt models to run
    "retry_delay": timedelta(minutes=1), # Retry after 1 minute
    "start_date": days_ago(1), # Start the process yesterday (for immediate execution)
    "schedule_interval": "*/5 * * * *", # Schedule the DAG every 5 minutes
}

# Initiate a DAG
with DAG(
    dag_id="cartelis-test",
    default_args=default_args,
) as dag:

		# Airbyte data integration task
    sync_source_destination = AirbyteTriggerSyncOperator(
        task_id="airflow-airbyte-sync",
        airbyte_conn_id="airflow-airbyte", # Name of the Airflow-Airbyte connection in Airflow
        connection_id="airbyte_connection_id", # ID of the airbyte connection between the file and the PostgreSQL
    )

		# dbt data transformation task
    dbt_run = DbtRunOperator(
        task_id="dbt_run",
    )
    sync_source_destination >> dbt_run